### Portail meteo.data - Téléchargement-affichage-Extraction des "dernières" données (latest) QUOTIDIENNES de Météo-France (RR-T-Vent)
Ce script ntraite uniquement le fichier "RR-T-vent" (Précipitations, Température, Vent) à l'exclusion du fichier "autres-parametres" contenant notamment l'ETP.<br>
Une connexion internet est nécessaire pour accéder aux archives des données à l'url ci-dessous.

- Les "dernières" données (latest) correspondent aux fichiers mis à jour quotidiennement, et qui vont du mois de janvier de l'année précédente au mois en cours même partiel.
- Les fichiers quotidiens sont téléchargés et décompressés automatiquement, pour plusieurs départements si besoin
- Les graphiques chronologiques QUOTIDIEN & MENSUEL sont tracés pour le paramètre Précipitations RR des postes choisis par l'utilisateur
- Un fichier excel rassemble:
    - Les données intégrales (tous paramètres de la période "Latest" pour tous les postes des départements concernés).
    - la comparaison des précipitations pour les postes et la période choisis, avec les graphiques quotidien & mensuel

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)<br>
Fichier CSV descriptif champs: https://www.data.gouv.fr/fr/datasets/r/6a8df7e9-45ff-445d-9260-6c65475dda86

Utilisez mon autre script pour visualiser sous forme de carte la liste des postes météorologiques fournie par Météo-France sous forme de fichier JSON https://meteo.data.gouv.fr/https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922

Auteur: https://github.com/loicduffar

##### 1) Télécharge les archives ZIP & décompresse les fichiers CSV LATEST (DERNIERE PERIODE DISPONIBLE)

- Définir les chemins d'enregistrement pour les archives GZ et pour les fichier CSV décompressés
- Définir les postes à interroger par leur code (voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922)
- Les archives LATEST sont automatiquement téléchargées et décompressées


In [120]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12
# Télécharge les les archives ZIP & et décompresse les fichiers CSV LATEST (DERNIERE PERIODE DISPONIBLE)
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Personalisation ====================
# Chemin d'enregistrement des archives gz et des fichiers CSV décompressés
folder_gz= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\Q"
folder_csv= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q"

# Numéro des Postes météo souhaités (chaine de 8 caractères) - voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922
# LES DEPARTEMENTS CORRESPONDANTS DOIVENT ËTRE PRESENTS DANS LA LISTE DES URLS (voir ======= Initialisation ==========)
postes= [
        '04088001', '04039001', '04230001',                  # Forcalquier, Castellane, Valensole \
        # '05046001',                                           # Embrun \
        # '06088001', '06029001',                               # Nice, Cannes \
        '13001009', '13111002', '13103001','13055001',        # Aix en Provence, Vauvenargue, Salon, Marseille, \
        '83031001', '83061001', '83137001',                   # Le Luc, Fréjus, Toulon \
        '84003002', '84009002',                               # # Apt-Viton, Bastide des Jourdans \
        ]                               

# ================ Extrait du fichier data CSV ============= (les colonnes sont en réalité séparées par des points-virgules)
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMMJJ	RR	QRR	TN	QTN	HTN	QHTN	TX	QTX	HTX	QHTX	TM	QTM	TNTXM	QTNTXM	TAMPLI	QTAMPLI
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220101	0	1	4.6	1	820	9	9.4	1	1424	9	7.9	1	7	1	4.8	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220102	0	1	8.3	1	646	9	11.1	1	1145	9	9.1	1	9.7	1	2.8	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220103	0.2	1	4.6	1	655	9	13.5	1	1452	9	7.4	1	9.1	1	8.9	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220104	1.4	1	4.5	1	2348	9	13.7	1	1123	9	10.1	1	9.1	1	9.2	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220105	0	1	4.6	1	1742	9	8.7	1	601	9	6.8	1	6.7	1	4.1	1

# ================ Extrait du fichier CSV Descriptif de quelques paramètres (les plus souvent utiles, précipitations et températures) =============(les colonnes sont en réalité séparées par ":")
# NUM_POSTE   	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   	 nom usuel du poste
# LAT         	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON         	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJ    	 date de la mesure (année mois jour)
# RR          	 quantité de précipitation tombée en 24 heures (de 06h FU le jour J à 06h FU le jour J+1). La valeur relevée à J+1 est affectée au jour J (en mm et 1/10)
# TN          	 température minimale sous abri (en °C et 1/10)
# HTN         	 heure de TN (hhmm)
# TX          	 température maximale sous abri (en °C et 1/10)
# HTX         	 heure de TX (hhmm)
# TM          	 moyenne quotidienne des températures horaires sous abri (en °C et 1/10)
# TNTXM       	 moyenne quotidienne (TN+TX)/2 (en °C et 1/10)
# TAMPLI      	 amplitude thermique quotidienne -  écart entre TX et TN quotidiens (TX-TN) (en °C et 1/10)
# TNSOL       	 température quotidienne minimale à 10 cm au-dessus du sol (en °C et 1/10)
# TN50        	 température quotidienne minimale à 50 cm au-dessus du sol (en °C et 1/10)
# DG          	 durée de gel sous abri (T ≤ 0°C) (en mn)

# ================ Initialisation ====================
# Structure du nom des fichiers de données QUOTIDIENNES (1 fichier par département dont le numéro sera ajouté automatiquement au début et à la fin du template ci-dessous)
template_start= 'Q_'
template_end= '_latest-2022-2023_RR-T-Vent.csv'

# ---------------- urls de téléchargement des archives PAR DEPARTEMENT des dernières données depuis janvier de l'année précédente https://meteo.data.gouv.fr/ (Ajouter d'autres départements si besoin)
url_desc = "https://www.data.gouv.fr/fr/datasets/r/6a8df7e9-45ff-445d-9260-6c65475dda86"
urls= dict()
urls['04']= "https://www.data.gouv.fr/fr/datasets/r/52355223-df85-49a9-9031-14b8a604c28c" 
urls['05']= 'https://www.data.gouv.fr/fr/datasets/r/27bfba71-f9d2-4cbb-a784-7a893d933485'
urls['06']= 'https://www.data.gouv.fr/fr/datasets/r/6d848c25-210b-40a4-b3c2-a4084bf660a7'
urls['13']= 'https://www.data.gouv.fr/fr/datasets/r/eb4d0600-e90b-4517-a429-599ed13dbae0'
urls['83']= 'https://www.data.gouv.fr/fr/datasets/r/5889ea7c-e285-46cb-8ea6-1c317fa4dada'
urls['84']= 'https://www.data.gouv.fr/fr/datasets/r/b47bd200-0ba7-419c-9385-4ce8ff8c199c'
# print(urls)

# ---------------- Définition des fonctions
def convert_to_date(aaaammdd):
    return pd.to_datetime(str(aaaammdd), format='%Y%m%d', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
        # print("Téléchargement terminé")
    else:
        print("Fichier non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier non trouvé: ", file)

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    print('Lecture: ', file)
    df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMMJJ':str}, parse_dates=['AAAAMMJJ'], date_parser= convert_to_date)
    return df

# ================ Traitement ====================
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"))

# Lit en ligne le fichier "Q_descriptif_champs_RR-T-Vent.csv" de description des champs
# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc = pd.read_csv(url_desc, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc.index= df_desc.index.str.strip()
# display(df_desc)

# fait la liste des départements concernés et garde les 2 premiers chiffres uniques
# On récupère les 2 premiers chiffres de chaque poste
departements = []
for poste in postes:
    departements.append(poste[:2])
# On ne garde que départements uniques
departements = list(set(departements)) # set() ignore automatiquement les doublons (ce qui évite de passer par array numpy pour utiliser la fonction np.unique() )
# On trie la liste
departements.sort()
# On affiche la liste
print('Départements concernés: ', departements)

# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
for i, departement in enumerate(departements):
    # On récupère l'url
    url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
    filename = f"{template_start}{departement}{template_end}"
    download_file(url, filename)
    decompress_gz(filename)
    # Lecture du fichier CSV dans un dataframe pandas
    df_departement= read_csv(filename)
    if i== 0:
        # initialisation du dataframe final df avec les données du premier département        
        df= df_departement
    else:
        df= pd.concat([df, df_departement])        

df['NUM_POSTE'] = df['NUM_POSTE'].str.strip()
# renome la colonne "AAAMMJJ" en "DATE"
df.rename(columns={'AAAAMMJJ':'DATE'}, inplace=True)
display(df)

2023-12-22 17:00
Départements concernés:  ['04', '13', '83', '84']
Téléchargement:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\Q\Q_04_latest-2022-2023_RR-T-Vent.csv.gz
Décompression X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_04_latest-2022-2023_RR-T-Vent.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_04_latest-2022-2023_RR-T-Vent.csv
Téléchargement:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\Q\Q_13_latest-2022-2023_RR-T-Vent.csv.gz
Décompression X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_13_latest-2022-2023_RR-T-Vent.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,DATE,RR,QRR,TN,QTN,...,DXI2,QDXI2,HXI2,QHXI2,FXI3S,QFXI3S,DXI3S,QDXI3S,HXI3S,QHXI3S
0,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-01,0.0,1.0,1.1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-02,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-03,0.0,1.0,-0.4,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-04,1.6,1.0,0.2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-05,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12937,84150001,VISAN,44.336667,4.905500,141,2023-12-16,0.0,1.0,5.4,1.0,...,NaN,NaN,NaN,NaN,15.0,1.0,NaN,NaN,728.0,9.0
12938,84150001,VISAN,44.336667,4.905500,141,2023-12-17,0.0,1.0,-1.7,1.0,...,NaN,NaN,NaN,NaN,9.1,1.0,NaN,NaN,37.0,9.0
12939,84150001,VISAN,44.336667,4.905500,141,2023-12-18,0.2,1.0,-3.4,1.0,...,NaN,NaN,NaN,NaN,3.1,1.0,NaN,NaN,1258.0,9.0
12940,84150001,VISAN,44.336667,4.905500,141,2023-12-19,0.2,1.0,-2.6,1.0,...,NaN,NaN,NaN,NaN,12.7,1.0,NaN,NaN,2319.0,9.0


##### 2) Tracé du graphique des précipitations qutotidiennes

Les dates de début et de fin du graphique sont définies comme suit:
- Laissez le script déterminer automatiquement les 2 derniers mois
- ou définir la date de début et de fin,
- ou None pour respectivement le début et la fin du fichier LATEST (premier jour de l'année précédente et veille du téléchargement)

NB: on peut choisir le paramètre mais la présentation actuelle est spécialement adaptée aux précipitations

In [121]:
# trace un graphique plotly de df avec la colonne 'DATE' en abscisse et la colonne 'RR' en ordonnée
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import calendar
import numpy as np

#-------------- Personalisation du graphique
# Paramètre à représenter (la présentation actuelle est spécialement adaptée aux précipitations RR)
param= 'RR'

# Période à représenter (None pour le début ou la fin du fichier par défaut)
start_date= None
# Calculez start_date comme le premier jour du mois précédent
if now.month == 1:  # Si le mois actuel est janvier
    start_date = datetime.datetime(now.year - 1, 12, 1)
else:  # Pour tous les autres mois
    start_date = datetime.datetime(now.year, now.month - 1, 1)
# start_date= datetime.datetime(2023, 8, 1)

end_date= datetime.datetime(now.year, now.month, now.day)

#-------------- initialisation des paramètres du graphique
# Période par défaut if start_date= None ou  end_date= None
if start_date is None:
    start_date= df['DATE'].min()
if end_date is None:
    end_date= df['DATE'].max()
# Définit le premier jour du premier mois et dernier jour du dernier mois (pour une graduation clair de l'axe ddes dates)
start_date= datetime.datetime(start_date.year, start_date.month, 1)
end_date= datetime.datetime(end_date.year, end_date.month, 1) + datetime.timedelta(days= 30)

# filtre le dataframe sur les postes choisis
df_postes= df[df['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_postes= df_postes[(df_postes['DATE'] >= start_date) & (df_postes['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Set3
color_sequence_bar = plotly.colors.qualitative.Alphabet
color_sequence_bar = plotly.colors.qualitative.Dark24

# définit une séquence de couleurs pour les courbes cumulées
# color_sequence_scatter = color_sequence_bar[::-1]
# color_sequence_scatter = plotly.colors.qualitative.Set3
color_sequence_scatter = plotly.colors.qualitative.Alphabet
# color_sequence_scatter = plotly.colors.qualitative.Dark24

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)],  # Utilisez la couleur correspondante de la séquence
                                    line= dict(color='rgb(100,100,100)', width=1),  # Ligne de contour des barres
                                    ),
                        # hovertemplate = 
                            # '<i>Date</i>: %{x}' +
                            # '<br><b>RR</b>: %{y}<br>',
                            # text = df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]                                    
                         ),
                row= i+1, col= 1
                )
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]
                                        ), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                    # hovertemplate = 
                    #         '<br><b>Cumulé</b>: %{y}<br>'                                      
                             ), 
                 row=i+1, col=1, secondary_y= True,
                 )

fig.update_layout(title_text= 'meteo.data - ' + "Précipitations qutotidiennes (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                    )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot (empêche la graduation des dates de tous les subplots)

# Définition générale des axes
max_value= df_postes[param].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()[param].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# GRADUATION de l'axe des DATES ........................................
if end_date - start_date > datetime.timedelta(days= 365): # plus d'un an
    tickformat= '%-d%b\n%Y'
    dtick= 'M2'
else: # Moins
    tickformat= '%d %b\n%Y'
    dtick= 'M1' # (pour une raison inconnue,'D10' ne marche pas du tout et le nombre de milisecondes ne marche qu'après la 2ème graduation et )
# ............................... Premiers subplots
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ............................... Dernier subplot
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x', range=[start_date, end_date])

# GRILLE.................................................................
# .................. verticales 
# .............................. Premiers subplots INUTILE ?
for i in range(1, len(postes) + 1):
    fig.update_xaxes(showgrid= True, tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1)
# .............................. Dernier subplot TOUS ?
fig.update_xaxes(showgrid= True, tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1)
# .................. horizontales
# ............................... Axe secondaire
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph_quot= "meteo.data QUOT"
fig.write_html(os.path.join(folder_csv, file_graph_quot + template_end[:-4] + ".html"))
# sauvegarde le graphique dans un fichier png
fig.write_image(os.path.join(folder_csv, file_graph_quot + template_end[:-4] + ".png"))

#-------------- Pluies quotitdiennes SUR LA DUREEE COMPLETE et cumuls mensuelles pour les postes sélectionnés
# df_pj = df_postes.pivot(index='DATE', columns='NOM_USUEL', values='RR')
df_pj = df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values='RR')
def sum_or_nan(s):
    return s.sum() if s.count() == s.size else np.nan
# calcul le cumul mensuel de RR pour chaque poste en affectant NaN aux mois incomplets en appliquant la date du premier jour du mois
df_pm= df_pj.resample('MS').apply(sum_or_nan)

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
# imprimer un avertissement si la dernière date du dataframe est antérieure au dernier jour du mois en cours
# calculer la date du dernier jour du mois en cours en tenant compte des jours à 28, 29 et 30 jours
last_date= datetime.datetime(now.year, now.month, calendar.monthrange(now.year, now.month)[1])
titre_pm= 'Cumuls mensuels de pluie des postes et de la période choisis: '
if np.datetime64(df['DATE'].values[-1]) < np.datetime64(last_date):
    titre_pm= titre_pm + 'Attention le cumul du dernier mois est incomplet à ce jour'

print(titre_pm)
display(df_pm[(df_pm.index >= start_date) & (df_pm.index <= end_date)])
fig.show()
titre_pj= 'Précipitations quotidiennes des postes et de la période choisis'
print(titre_pj)
display(df_pj[(df_pj.index >= start_date) & (df_pj.index <= end_date)])

2023-12-22 17:00
Cumuls mensuels de pluie des postes et de la période choisis: Attention le cumul du dernier mois est incomplet à ce jour


NOM_USUEL,AIX EN PROVENCE,APT-VITON,CASTELLANE,FORCALQUIER,FREJUS,LA BASTIDE DES JOURDANS,LE LUC,MARSEILLE-OBS,SALON DE PROVENCE,TOULON,VALENSOLE,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-11-01,30.2,37.6,102.2,63.1,20.4,22.0,21.7,23.2,30.8,25.8,36.4,41.8
2023-12-01,42.6,60.4,76.3,NaN,50.6,42.6,53.8,28.3,35.4,65.4,56.8,71.0


Précipitations quotidiennes des postes et de la période choisis


NOM_USUEL,AIX EN PROVENCE,APT-VITON,CASTELLANE,FORCALQUIER,FREJUS,LA BASTIDE DES JOURDANS,LE LUC,MARSEILLE-OBS,SALON DE PROVENCE,TOULON,VALENSOLE,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-11-01,0.0,0.6,5.6,2.0,0.6,0.2,2.0,0.8,0.0,0.0,0.8,1.4
2023-11-02,4.2,6.6,26.6,10.4,3.2,3.2,4.6,5.0,3.0,3.8,11.2,9.6
2023-11-03,0.6,2.2,0.4,1.8,0.0,0.6,0.0,1.6,0.4,2.0,1.4,0.2
2023-11-04,1.8,4.6,23.3,11.2,9.2,2.2,3.6,1.2,0.4,3.2,6.4,6.4
2023-11-05,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-06,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0
2023-11-07,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-08,0.0,0.4,0.4,0.3,0.0,0.0,0.0,0.0,0.6,0.0,0.2,0.2
2023-11-09,8.2,11.0,9.0,18.1,3.8,7.0,6.9,8.4,15.2,12.8,5.2,9.4


##### 3) Tracé du graphique des cumul mensuels de précipitations

Définir au préalable comme suit les dates de début et de fin du graphique:
- None pour prendre le début et la fin du fichier LATEST (premier jour de l'année précédente et veille du téléchargement)
- ou définir la date de début et de fin

NB: on peut choisir le paramètre mais la présentation actuelle est spécialement adaptée aux précipitations

In [122]:
# trace un graphique plotly de df avec la colonne 'DATE' en abscisse et la colonne 'RR' en ordonnée
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

#-------------- Personalisation du graphique 
# Paramètre à représenter (la présentation actuelle est spécialement adaptée aux précipitations RR)
param= 'RR'

# Période à représenter (None pour le début ou la fin du fichier par défaut)
start_date= None
# start_date= datetime.datetime(2022, 5, 1)

end_date= None
# end_date= datetime.datetime(now.year, now.month, now.day)

#-------------- initialisation des paramètres du graphique
# à partir de df_postes, génère un dataframe indexé sur 'NOM_USUEL' avec une colonne 'NUM_POSTE' et
df_desc_postes= df_postes[['NUM_POSTE', 'NOM_USUEL']].drop_duplicates().set_index('NOM_USUEL')

# réorganise le dataframe df_pm (SUR LA DUREE COMPLETE) avec les postes en lignes et les mois lignes et une seule colonne 'RR'
df_pm2= df_pm.stack().reset_index(name='RR')

# ajoute à df_pm2 une colonne 'NUM_POSTE' avec le numéro du poste
df_pm2= df_pm2.join(df_desc_postes, on='NOM_USUEL')

# Période par défaut if start_date= None ou  end_date= None
if start_date is None:
    start_date= df_pm2.reset_index()['DATE'].min()
if end_date is None:
    end_date= df_pm2.reset_index()['DATE'].max()

# filtre le dataframe sur les postes choisis
df_postes= df_pm2[df_pm2['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_postes= df_postes[(df_postes['DATE'] >= start_date) & (df_postes['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Set3
# color_sequence_scatter = plotly.colors.qualitative.Dark24

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)]),  # Utilisez la couleur correspondante de la séquence
                         ),
                row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'),
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                             mode='lines' 
                             ), 
                 row=i+1, col=1, secondary_y= True)

fig.update_layout(title_text= 'meteo.data - ' + "Précipitations mensuelles (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                  )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot

# Définition générale des axes
max_value= df_postes[param].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()[param].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# Graduations de l'axe des DATES ...............................................................
if end_date - start_date > datetime.timedelta(days= 365): # Plus d'un an
    tickformat= '%b\n%Y'
    dtick= 'M6'
    showgrid_x= True
else: # Moins d'un an
    tickformat= '%b %Y'
    dtick= 'M1'
    showgrid_x= False
# ...............................Premiers subplots
# ................................................ en fonction de la longueur de la période
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ...............................Dernier subplot
# ................................................ en fonction de la longueur de la période
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x')
#                                                SANS tenir compte de la longueur de la période
# fig.update_xaxes(tickformat= '%b\n%Y', tickmode= 'linear', dtick= 'M1', row=len(postes), col=1, matches='x')

# GRILLE .............................................................................
# .................. verticales...
fig.update_xaxes(showgrid= showgrid_x, gridwidth= 1, gridcolor= 'white')
# .................. horizontales
#                                Axe secondaire:
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph_mens= "meteo.data MENS"
fig.write_html(os.path.join(folder_csv, file_graph_mens + template_end[:-4] + ".html"))
# sauvegarde le graphique dans un fichier png
fig.write_image(os.path.join(folder_csv, file_graph_mens + template_end[:-4] + ".png"))

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
# afficher RR pour df_postes avec les postes en colonnes et les dates en index
fig.show()
print('Précipitations mensuelles LATEST (mm)')
# df_pm2= df_postes.pivot(index= 'DATE', columns= 'NOM_USUEL', values= 'RR')
display(df_postes.pivot(index= 'DATE', columns= 'NOM_USUEL', values= 'RR'))

2023-12-22 17:01


Précipitations mensuelles LATEST (mm)


NOM_USUEL,AIX EN PROVENCE,APT-VITON,CASTELLANE,FORCALQUIER,FREJUS,LA BASTIDE DES JOURDANS,LE LUC,MARSEILLE-OBS,SALON DE PROVENCE,TOULON,VALENSOLE,VAUVENARGUES
DATE,,,,,,,,,,,,
2022-01-01,3.6,1.8,5.6,3.5,2.4,3.4,0.8,0.4,2.0,2.8,4.8,1.0
2022-02-01,26.1,23.7,35.4,24.8,36.2,18.4,31.9,19.9,38.2,27.0,15.0,24.7
2022-03-01,6.6,11.6,13.4,6.6,30.4,6.4,28.2,9.4,6.6,16.4,8.4,10.4
2022-04-01,27.0,53.7,65.6,44.4,40.4,41.5,45.8,19.3,25.6,26.2,32.6,38.9
2022-05-01,26.0,47.3,29.4,28.5,3.8,NaN,38.1,29.5,7.6,28.6,29.0,44.7
2022-06-01,7.0,12.1,47.1,20.4,43.4,10.5,16.6,8.8,17.2,24.5,15.8,11.0
2022-07-01,0.2,1.4,8.6,3.6,0.2,1.0,0.6,0.0,0.2,0.2,0.0,0.4
2022-08-01,37.7,19.1,54.1,93.6,33.1,36.8,54.1,63.2,51.1,40.9,56.9,21.0
2022-09-01,49.0,36.2,67.1,83.2,44.0,43.2,41.1,62.2,47.1,64.7,78.0,69.6


##### 4) Sauvegarde les données intégrales dans un fichier Excel (tous les paramètres pour tous les postes des départements concernés)
Le fichier est sauvegardé dans le même dossier local que les fichiers CSV décompressés

In [123]:
# Sauvegarde le dataframe dans un fichier Excel

print(now.strftime("%Y-%m-%d %H:%M"))
print('Sauvegarde en cours, patientez...')

file= os.path.join(folder_csv, "meteo.data QUOT"+ '_' + '-'.join(departements) + template_end[:-4] + ".xlsx")
writer = pd.ExcelWriter(file, engine='xlsxwriter')
df.to_excel(writer, sheet_name='donnée complètes', index=False)

# ajoute des onglets exel avec df_pj et df_pm
df_pj.to_excel(writer, sheet_name='PJ Sélection', index=True, startrow= 2)
# ajoute un texte
workbook  = writer.book
worksheet = writer.sheets['PJ Sélection']
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_pj)

df_pm.to_excel(writer, sheet_name='PM Sélection', index=True, startrow= 2)
# ajoute un texte
worksheet = writer.sheets['PM Sélection']
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_pm)

# ajoute un onglet excel et enregistre une image du graphique
workbook  = writer.book
worksheet = workbook.add_worksheet('graph')
worksheet.insert_image('A1', os.path.join(folder_csv, file_graph_quot + template_end[:-4] + ".png"))
worksheet.insert_image('Q1', os.path.join(folder_csv, file_graph_mens + template_end[:-4] + ".png"))

writer.save()

print('Ficher Excel sauvegardé: ', file)

2023-12-22 17:00
Sauvegarde en cours, patientez...
Ficher Excel sauvegardé:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\meteo.data QUOT_04-13-83-84_latest-2022-2023_RR-T-Vent.xlsx
